## Coursera project-The Battle of Neighborhoods

# Table of contents

#### 1.Introduction
#### 2.Data Description
#### 3.Methodology
#### 4.Code
#### 5.Results
#### 6.Discussion

# 1.Introduction
A world-wide event planning and management company is currently working on a project to organize an international event for one week in Washington State Convention Center (WSCC), Seattle, WA. The Washington State Convention Center (WSCC) is a convention center in Seattle, Washington, United States. It consists of several exhibition halls and meeting rooms in buildings along Pike Street in Downtown Seattle. Part of the complex straddles Interstate 5 and connects with Freeway Park. The convention center was planned in the late 1970s and funded through $90 million in bonds issued by the state legislature.The requests are including finding list of hotels of residence, restaurants and coffee shops close to the conference center. This project will help the event planner to find the list of restaurants, coffee shops and hotels. Since most of the people are from Italy, the planner also want to know the list of Italian restaurants near WSCC.

# 2.Data Description
The data used in this project is provided by Foursquare location data. I will construct a URL to send a request to the API to search for a specific to explore a geographical location, and to get trending venues around a location. The data are grouped by landscape area, and each area included the information about this area and all information about hotels, restaurants and shopping malls in this area. 

# 3.Methodology

The convention center is located along Pike Street between 7th Avenue and Hubbell Place in Downtown Seattle. The complex straddles Pike Street, 8th Avenue, and Interstate 5, stretching from Union Street in the south to Pine Street in the north. It is adjacent to several downtown hotels and the Pike–Pine retail corridor, which includes Westlake Center and Pacific Place.The convention center is served by several transit routes, including Westlake station in the Downtown Seattle Transit Tunnel; the tunnel carries suburban buses and Link light rail trains, which serve Seattle–Tacoma International Airport. The facility is also served by buses on surface streets, including several trolleybus routes. 

In this project I will direct our efforts on detecting areas of WSCC for the people attending the event for one week. The public transposition is convenient in Seattle downtown. I will limit my analysis to area ~500m around WSCC.

In first step I will collect and clean the required data: location of restaurants, hotels and coffee shops within 6km from WSCC. Second step in my analysis will be calculation and exploration of 'Italian shops density' across different areas of Seattle. I will create clusters (using k-means clustering) of those locations to identify general.

# 4.Code

## Step 1: Find the restaurants, hotels and coffee shops near by Washington State Convention Center

In [1]:
import requests # to handle requests
import pandas as pd # for data analsysis
import numpy as np # to handle data in a vectorized manner

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
#tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.49   | 

In [2]:
ClIENT_ID = 'NJBI1NENJYENG0SGJTTS1RFHOKSQEHBMWXQHPRLSLI0XU0OX' # your Foursquare ID
ClIENT_SECRET = 'E5GJDJ4JT02EDEH0PVGKF1ZN0NJCRTLXPJFMYPP0OJ4XAKLA' # your Foursquare Secret
VERSION = '20180604'
LIMIT =30
print('Your credentails:')
print('Foursquare_ID: ' + ClIENT_ID)
print('Foursquare_Secret:' + ClIENT_SECRET)

Your credentails:
Foursquare_ID: NJBI1NENJYENG0SGJTTS1RFHOKSQEHBMWXQHPRLSLI0XU0OX
Foursquare_Secret:E5GJDJ4JT02EDEH0PVGKF1ZN0NJCRTLXPJFMYPP0OJ4XAKLA


## Define Washington State Convention Center, Seattle and get its latitude and longitude

In [3]:
# define the conference center and get its latitude & longitude 
city = 'Washington State Convention Center'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

47.61175385 -122.331562750662


## Get a list of hotels

In [4]:
# search for hotels
search_query = 'Hotel'
radius = 500

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=NJBI1NENJYENG0SGJTTS1RFHOKSQEHBMWXQHPRLSLI0XU0OX&client_secret=E5GJDJ4JT02EDEH0PVGKF1ZN0NJCRTLXPJFMYPP0OJ4XAKLA&ll=47.61175385,-122.331562750662&v=20180604&query=Hotel&radius=500&limit=30'

In [5]:
# Send the GET Request and examine the results
results = requests.get(url).json()
#results

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,...,location.country,location.formattedAddress,venuePage.id,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,location.neighborhood
0,4a089147f964a520ea731fe3,Sheraton Seattle Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1565642129,False,1400 6th Ave,at Pike St,47.610697,-122.333895,"[{'label': 'display', 'lat': 47.6106973, 'lng'...",...,United States,"[1400 6th Ave (at Pike St), Seattle, WA 98101,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,49e39fa6f964a520a2621fe3,Renaissance Seattle Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1565642129,False,515 Madison St,at 5th Ave,47.606667,-122.330685,"[{'label': 'display', 'lat': 47.60666749906805...",...,United States,"[515 Madison St (at 5th Ave), Seattle, WA 9810...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4a4d485cf964a520d2ad1fe3,Fairmont Olympic Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1565642129,False,411 University St,at 4th Ave,47.608328,-122.334211,"[{'label': 'display', 'lat': 47.60832812585564...",...,United States,"[411 University St (at 4th Ave), Seattle, WA 9...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4a4b973af964a52078ac1fe3,Kimpton Hotel Vintage Seattle,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1565642129,False,1100 5th Ave,at Spring St,47.607657,-122.332345,"[{'label': 'display', 'lat': 47.607657, 'lng':...",...,United States,"[1100 5th Ave (at Spring St), Seattle, WA 9810...",177958104,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,457945ebf964a520cc3e1fe3,Kimpton Hotel Monaco Seattle,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1565642129,False,1101 4th Ave,at Spring St,47.606932,-122.334100,"[{'label': 'display', 'lat': 47.606932, 'lng':...",...,United States,"[1101 4th Ave (at Spring St), Seattle, WA 9810...",358542026,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Clean the Hotel dataframe

In [6]:
# keep only columns that include venue name, and anything that is associated with location
clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_dataframe = dataframe.loc[:,clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]

clean_dataframe.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Sheraton Seattle Hotel,Hotel,1400 6th Ave,at Pike St,47.610697,-122.333895,"[{'label': 'display', 'lat': 47.6106973, 'lng'...",210,98101,US,Seattle,WA,United States,"[1400 6th Ave (at Pike St), Seattle, WA 98101,...",NaN,4a089147f964a520ea731fe3
1,Renaissance Seattle Hotel,Hotel,515 Madison St,at 5th Ave,47.606667,-122.330685,"[{'label': 'display', 'lat': 47.60666749906805...",570,98104,US,Seattle,WA,United States,"[515 Madison St (at 5th Ave), Seattle, WA 9810...",NaN,49e39fa6f964a520a2621fe3
2,Fairmont Olympic Hotel,Hotel,411 University St,at 4th Ave,47.608328,-122.334211,"[{'label': 'display', 'lat': 47.60832812585564...",430,98101,US,Seattle,WA,United States,"[411 University St (at 4th Ave), Seattle, WA 9...",NaN,4a4d485cf964a520d2ad1fe3
3,Kimpton Hotel Vintage Seattle,Hotel,1100 5th Ave,at Spring St,47.607657,-122.332345,"[{'label': 'display', 'lat': 47.607657, 'lng':...",459,98101,US,Seattle,WA,United States,"[1100 5th Ave (at Spring St), Seattle, WA 9810...",NaN,4a4b973af964a52078ac1fe3
4,Kimpton Hotel Monaco Seattle,Hotel,1101 4th Ave,at Spring St,47.606932,-122.334100,"[{'label': 'display', 'lat': 47.606932, 'lng':...",569,98101,US,Seattle,WA,United States,"[1101 4th Ave (at Spring St), Seattle, WA 9810...",NaN,457945ebf964a520cc3e1fe3


In [7]:
# delete unnecessary columns
clean_dataframe2= clean_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id', 'neighborhood'], axis=1)
clean_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Seattle Hotel,Hotel,1400 6th Ave,47.610697,-122.333895,98101,WA
1,Renaissance Seattle Hotel,Hotel,515 Madison St,47.606667,-122.330685,98104,WA
2,Fairmont Olympic Hotel,Hotel,411 University St,47.608328,-122.334211,98101,WA
3,Kimpton Hotel Vintage Seattle,Hotel,1100 5th Ave,47.607657,-122.332345,98101,WA
4,Kimpton Hotel Monaco Seattle,Hotel,1101 4th Ave,47.606932,-122.334100,98101,WA
5,Hotel Max,Hotel,620 Stewart St,47.614220,-122.336460,98101,WA
6,The Paramount Hotel Seattle,Hotel,724 Pine St,47.613007,-122.333485,98101,WA
7,Executive Hotel Pacific,Hotel,400 Spring St,47.606935,-122.333325,98104,WA
8,Hotel Sorrento,Hotel,900 Madison St,47.608885,-122.326757,98104,WA
9,Mayflower Park Hotel,Hotel,405 Olive Way,47.612293,-122.338419,98101,WA


In [8]:
# delete rows with none values
clean_dataframe3 = clean_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_dataframe3

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Seattle Hotel,Hotel,1400 6th Ave,47.610697,-122.333895,98101,WA
1,Renaissance Seattle Hotel,Hotel,515 Madison St,47.606667,-122.330685,98104,WA
2,Fairmont Olympic Hotel,Hotel,411 University St,47.608328,-122.334211,98101,WA
3,Kimpton Hotel Vintage Seattle,Hotel,1100 5th Ave,47.607657,-122.332345,98101,WA
4,Kimpton Hotel Monaco Seattle,Hotel,1101 4th Ave,47.606932,-122.334100,98101,WA
5,Hotel Max,Hotel,620 Stewart St,47.614220,-122.336460,98101,WA
6,The Paramount Hotel Seattle,Hotel,724 Pine St,47.613007,-122.333485,98101,WA
7,Executive Hotel Pacific,Hotel,400 Spring St,47.606935,-122.333325,98104,WA
8,Hotel Sorrento,Hotel,900 Madison St,47.608885,-122.326757,98104,WA
9,Mayflower Park Hotel,Hotel,405 Olive Way,47.612293,-122.338419,98101,WA


In [9]:
# delete rows which its category is not Hotel or Event Space
array= ['Hotel']
df_Hotel= clean_dataframe3.loc[clean_dataframe3['categories'].isin(array)]
df_Hotel

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Seattle Hotel,Hotel,1400 6th Ave,47.610697,-122.333895,98101,WA
1,Renaissance Seattle Hotel,Hotel,515 Madison St,47.606667,-122.330685,98104,WA
2,Fairmont Olympic Hotel,Hotel,411 University St,47.608328,-122.334211,98101,WA
3,Kimpton Hotel Vintage Seattle,Hotel,1100 5th Ave,47.607657,-122.332345,98101,WA
4,Kimpton Hotel Monaco Seattle,Hotel,1101 4th Ave,47.606932,-122.334100,98101,WA
5,Hotel Max,Hotel,620 Stewart St,47.614220,-122.336460,98101,WA
6,The Paramount Hotel Seattle,Hotel,724 Pine St,47.613007,-122.333485,98101,WA
7,Executive Hotel Pacific,Hotel,400 Spring St,47.606935,-122.333325,98104,WA
8,Hotel Sorrento,Hotel,900 Madison St,47.608885,-122.326757,98104,WA
9,Mayflower Park Hotel,Hotel,405 Olive Way,47.612293,-122.338419,98101,WA


## Get a list of restaurants

In [10]:
# search for Restaurants
search_query = 'Restaurant'
radius = 500

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=NJBI1NENJYENG0SGJTTS1RFHOKSQEHBMWXQHPRLSLI0XU0OX&client_secret=E5GJDJ4JT02EDEH0PVGKF1ZN0NJCRTLXPJFMYPP0OJ4XAKLA&ll=47.61175385,-122.331562750662&v=20180604&query=Restaurant&radius=500&limit=30'

In [11]:
# Send the GET Request and examine the results
Rresults = requests.get(url).json()
#Rresults

# assign relevant part of JSON to venues
venues = Rresults['response']['venues']

# tranform venues into a dataframe
Restaurant_dataframe = json_normalize(venues)
Restaurant_dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,...,location.country,location.formattedAddress,venuePage.id,location.neighborhood,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name
0,4a9c2ec7f964a520523620e3,Saigon Vietnamese Restaurant II,"[{'id': '4bf58dd8d48988d14a941735', 'name': 'V...",v-1565642130,False,1529 6th Ave,at Pine St,47.612039,-122.335397,"[{'label': 'display', 'lat': 47.61203898469434...",...,United States,"[1529 6th Ave (at Pine St), Seattle, WA 98101,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,51f86c01498e5f3d48939878,Retro Restaurant And Lounge,"[{'id': '4bf58dd8d48988d121941735', 'name': 'L...",v-1565642130,False,216 Stewart St,Stewart Street and 3rd Ave,47.611369,-122.339823,"[{'label': 'display', 'lat': 47.61136890490712...",...,United States,"[216 Stewart St (Stewart Street and 3rd Ave), ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4b0b066bf964a520bf2b23e3,Thai Ginger Restaurant,"[{'id': '4bf58dd8d48988d149941735', 'name': 'T...",v-1565642130,False,600 Pine St Fl 4,Level 4 Pacific Place,47.612759,-122.335104,"[{'label': 'display', 'lat': 47.61275897201381...",...,United States,"[600 Pine St Fl 4 (Level 4 Pacific Place), Sea...",225937215,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5542b3f9498ece80e0912ad8,Redtrees Bar & Restaurant,"[{'id': '4bf58dd8d48988d1d5941735', 'name': 'H...",v-1565642130,False,Hilton Seattle,NaN,47.609601,-122.333523,"[{'label': 'display', 'lat': 47.60960143910472...",...,United States,"[Hilton Seattle, Seattle, WA, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,40b13b00f964a52018f71ee3,Tango Restaurant & Lounge,"[{'id': '4bf58dd8d48988d107941735', 'name': 'A...",v-1565642130,False,1100 Pike St,at Boren Ave,47.613550,-122.329025,"[{'label': 'display', 'lat': 47.61355040316798...",...,United States,"[1100 Pike St (at Boren Ave), Seattle, WA 9810...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:

# keep only columns that include venue name, and anything that is associated with location
Restaurant_clean_columns = ['name', 'categories'] + [col for col in Restaurant_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Restaurant_dataframe = Restaurant_dataframe.loc[:,Restaurant_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']

# filter the category for each row
clean_Restaurant_dataframe['categories'] = clean_Restaurant_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Restaurant_dataframe.columns = [column.split('.')[-1] for column in clean_Restaurant_dataframe.columns]

clean_Restaurant_dataframe.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Saigon Vietnamese Restaurant II,Vietnamese Restaurant,1529 6th Ave,at Pine St,47.612039,-122.335397,"[{'label': 'display', 'lat': 47.61203898469434...",289,98101,US,Seattle,WA,United States,"[1529 6th Ave (at Pine St), Seattle, WA 98101,...",NaN,4a9c2ec7f964a520523620e3
1,Retro Restaurant And Lounge,Lounge,216 Stewart St,Stewart Street and 3rd Ave,47.611369,-122.339823,"[{'label': 'display', 'lat': 47.61136890490712...",621,98101,US,Seattle,WA,United States,"[216 Stewart St (Stewart Street and 3rd Ave), ...",NaN,51f86c01498e5f3d48939878
2,Thai Ginger Restaurant,Thai Restaurant,600 Pine St Fl 4,Level 4 Pacific Place,47.612759,-122.335104,"[{'label': 'display', 'lat': 47.61275897201381...",288,98101,US,Seattle,WA,United States,"[600 Pine St Fl 4 (Level 4 Pacific Place), Sea...",NaN,4b0b066bf964a520bf2b23e3
3,Redtrees Bar & Restaurant,Hotel Bar,Hilton Seattle,NaN,47.609601,-122.333523,"[{'label': 'display', 'lat': 47.60960143910472...",281,NaN,US,Seattle,WA,United States,"[Hilton Seattle, Seattle, WA, United States]",NaN,5542b3f9498ece80e0912ad8
4,Tango Restaurant & Lounge,Argentinian Restaurant,1100 Pike St,at Boren Ave,47.613550,-122.329025,"[{'label': 'display', 'lat': 47.61355040316798...",276,98101,US,Seattle,WA,United States,"[1100 Pike St (at Boren Ave), Seattle, WA 9810...",NaN,40b13b00f964a52018f71ee3


In [13]:

# delete unnecessary columns
clean_Restaurant_dataframe2= clean_Restaurant_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id', 'neighborhood'], axis=1)
clean_Restaurant_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Saigon Vietnamese Restaurant II,Vietnamese Restaurant,1529 6th Ave,47.612039,-122.335397,98101,WA
1,Retro Restaurant And Lounge,Lounge,216 Stewart St,47.611369,-122.339823,98101,WA
2,Thai Ginger Restaurant,Thai Restaurant,600 Pine St Fl 4,47.612759,-122.335104,98101,WA
3,Redtrees Bar & Restaurant,Hotel Bar,Hilton Seattle,47.609601,-122.333523,NaN,WA
4,Tango Restaurant & Lounge,Argentinian Restaurant,1100 Pike St,47.613550,-122.329025,98101,WA
5,Andaluca Restaurant,Restaurant,407 Olive Way,47.612257,-122.338135,98101,WA
6,Nordstrom Restaurant Division,Office,1600 7th Ave,47.613283,-122.333417,98101,WA
7,Kumagoro Restaurant,Food,1506 6th Ave,47.611190,-122.334570,98101,WA
8,Geneva Restaurant,Food,1106 8th Ave,47.609366,-122.329687,98101,WA
9,Daily Grill,Cocktail Bar,629 Pike St,47.611417,-122.333394,98101,WA


In [14]:
# delete rows with none values
df_Restaurant = clean_Restaurant_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Restaurant

,name,categories,address,lat,lng,postalCode,state
0,Saigon Vietnamese Restaurant II,Vietnamese Restaurant,1529 6th Ave,47.612039,-122.335397,98101,WA
1,Retro Restaurant And Lounge,Lounge,216 Stewart St,47.611369,-122.339823,98101,WA
2,Thai Ginger Restaurant,Thai Restaurant,600 Pine St Fl 4,47.612759,-122.335104,98101,WA
4,Tango Restaurant & Lounge,Argentinian Restaurant,1100 Pike St,47.613550,-122.329025,98101,WA
5,Andaluca Restaurant,Restaurant,407 Olive Way,47.612257,-122.338135,98101,WA
6,Nordstrom Restaurant Division,Office,1600 7th Ave,47.613283,-122.333417,98101,WA
7,Kumagoro Restaurant,Food,1506 6th Ave,47.611190,-122.334570,98101,WA
8,Geneva Restaurant,Food,1106 8th Ave,47.609366,-122.329687,98101,WA
9,Daily Grill,Cocktail Bar,629 Pike St,47.611417,-122.333394,98101,WA
10,Palomino,Italian Restaurant,1420 5th Ave,47.610410,-122.335191,98101,WA


In [15]:
# delete rows which its category is not Hotel or Event Space
array= ['Office']
df_Restaurant= df_Restaurant.loc[~df_Restaurant['categories'].isin (array)]
df_Restaurant

,name,categories,address,lat,lng,postalCode,state
0,Saigon Vietnamese Restaurant II,Vietnamese Restaurant,1529 6th Ave,47.612039,-122.335397,98101,WA
1,Retro Restaurant And Lounge,Lounge,216 Stewart St,47.611369,-122.339823,98101,WA
2,Thai Ginger Restaurant,Thai Restaurant,600 Pine St Fl 4,47.612759,-122.335104,98101,WA
4,Tango Restaurant & Lounge,Argentinian Restaurant,1100 Pike St,47.613550,-122.329025,98101,WA
5,Andaluca Restaurant,Restaurant,407 Olive Way,47.612257,-122.338135,98101,WA
7,Kumagoro Restaurant,Food,1506 6th Ave,47.611190,-122.334570,98101,WA
8,Geneva Restaurant,Food,1106 8th Ave,47.609366,-122.329687,98101,WA
9,Daily Grill,Cocktail Bar,629 Pike St,47.611417,-122.333394,98101,WA
10,Palomino,Italian Restaurant,1420 5th Ave,47.610410,-122.335191,98101,WA
11,Amicis Family Italian Restaurant,Italian Restaurant,1106 8th Ave,47.608721,-122.329190,98101,WA


## Get the list of coffee shop

In [16]:
# search for Parks
search_query = 'coffee'
radius = 500

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

url


'https://api.foursquare.com/v2/venues/search?client_id=NJBI1NENJYENG0SGJTTS1RFHOKSQEHBMWXQHPRLSLI0XU0OX&client_secret=E5GJDJ4JT02EDEH0PVGKF1ZN0NJCRTLXPJFMYPP0OJ4XAKLA&ll=47.61175385,-122.331562750662&v=20180604&query=coffee&radius=500&limit=30'

In [17]:
# Send the GET Request and examine the results
presults = requests.get(url).json()
#presults

# assign relevant part of JSON to venues
venues = presults['response']['venues']

# tranform venues into a dataframe
cof_dataframe = json_normalize(venues)
cof_dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,57e95a82498e0a3995a43e90,Anchorhead Coffee Co,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1565642132,False,1600 7th Ave Ste 105,Olive Way,47.613450,-122.334702,"[{'label': 'display', 'lat': 47.61344995548114...",301,98101,US,Seattle,WA,United States,"[1600 7th Ave Ste 105 (Olive Way), Seattle, WA...",NaN
1,5802b14d38fa7d4a74864fe9,Urban Coffee House,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1565642132,False,1900 4th Ave,NaN,47.612460,-122.338748,"[{'label': 'display', 'lat': 47.61246003668287...",544,98101,US,Seattle,WA,United States,"[1900 4th Ave, Seattle, WA 98101, United States]",NaN
2,4b05a7b3f964a52054e022e3,Tully's Coffee - Two Union,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1565642132,False,601 Union,NaN,47.610529,-122.331964,"[{'label': 'display', 'lat': 47.61052912009396...",139,98101,US,Seattle,WA,United States,"[601 Union, Seattle, WA 98101, United States]",NaN
3,4d6d4d94cb7a8cfa56685d79,Seattle's Best Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1565642132,False,NaN,in Sheraton Seattle Hotel,47.610726,-122.334468,"[{'label': 'display', 'lat': 47.61072552769955...",246,98101,US,Seattle,WA,United States,"[Seattle, WA 98101, United States]",NaN
4,56fecbc2498e190dff650557,Peet's Coffee & Tea,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1565642132,False,NaN,NaN,47.610033,-122.335019,"[{'label': 'display', 'lat': 47.610033, 'lng':...",322,NaN,US,Seattle,WA,United States,"[Seattle, WA, United States]",NaN


In [18]:

# keep only columns that include venue name, and anything that is associated with location
cof_clean_columns = ['name', 'categories'] + [col for col in cof_dataframe.columns if col.startswith('location.')]+ ['id']
clean_cof_dataframe = cof_dataframe.loc[:,cof_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list1 = row['categories']
    except:
        categories_list1 = row['venue.categories']
        
    if len(categories_list1) == 0:
        return None
    else:
        return categories_list1[0]['name']

# filter the category for each row
clean_cof_dataframe['categories'] = clean_cof_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_cof_dataframe.columns = [column.split('.')[-1] for column in clean_cof_dataframe.columns]

clean_cof_dataframe.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Anchorhead Coffee Co,Coffee Shop,1600 7th Ave Ste 105,Olive Way,47.613450,-122.334702,"[{'label': 'display', 'lat': 47.61344995548114...",301,98101,US,Seattle,WA,United States,"[1600 7th Ave Ste 105 (Olive Way), Seattle, WA...",57e95a82498e0a3995a43e90
1,Urban Coffee House,Coffee Shop,1900 4th Ave,NaN,47.612460,-122.338748,"[{'label': 'display', 'lat': 47.61246003668287...",544,98101,US,Seattle,WA,United States,"[1900 4th Ave, Seattle, WA 98101, United States]",5802b14d38fa7d4a74864fe9
2,Tully's Coffee - Two Union,Coffee Shop,601 Union,NaN,47.610529,-122.331964,"[{'label': 'display', 'lat': 47.61052912009396...",139,98101,US,Seattle,WA,United States,"[601 Union, Seattle, WA 98101, United States]",4b05a7b3f964a52054e022e3
3,Seattle's Best Coffee,Coffee Shop,NaN,in Sheraton Seattle Hotel,47.610726,-122.334468,"[{'label': 'display', 'lat': 47.61072552769955...",246,98101,US,Seattle,WA,United States,"[Seattle, WA 98101, United States]",4d6d4d94cb7a8cfa56685d79
4,Peet's Coffee & Tea,Coffee Shop,NaN,NaN,47.610033,-122.335019,"[{'label': 'display', 'lat': 47.610033, 'lng':...",322,NaN,US,Seattle,WA,United States,"[Seattle, WA, United States]",56fecbc2498e190dff650557


In [19]:
# delete unnecessary columns
clean_cof_dataframe2= clean_cof_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_cof_dataframe2


,name,categories,address,lat,lng,postalCode,state
0,Anchorhead Coffee Co,Coffee Shop,1600 7th Ave Ste 105,47.613450,-122.334702,98101,WA
1,Urban Coffee House,Coffee Shop,1900 4th Ave,47.612460,-122.338748,98101,WA
2,Tully's Coffee - Two Union,Coffee Shop,601 Union,47.610529,-122.331964,98101,WA
3,Seattle's Best Coffee,Coffee Shop,NaN,47.610726,-122.334468,98101,WA
4,Peet's Coffee & Tea,Coffee Shop,NaN,47.610033,-122.335019,NaN,WA
5,Victrola Coffee Roasters,Coffee Shop,1600 3rd Ave,47.610895,-122.338952,98101,WA
6,Cherry Street Coffee,Coffee Shop,509 Olive Way,47.612932,-122.337017,98101,WA
7,Voxx Coffee,Café,1200 6th Ave,47.608919,-122.332504,98101,WA
8,Made Coffee,Café,1531 7th Ave,47.612279,-122.334399,98101,WA
9,Retro Coffee,Coffee Shop,NaN,47.606662,-122.331461,98104,WA


In [20]:
# delete rows with none values
clean_cof_dataframe3 = clean_cof_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_cof_dataframe3


,name,categories,address,lat,lng,postalCode,state
0,Anchorhead Coffee Co,Coffee Shop,1600 7th Ave Ste 105,47.613450,-122.334702,98101,WA
1,Urban Coffee House,Coffee Shop,1900 4th Ave,47.612460,-122.338748,98101,WA
2,Tully's Coffee - Two Union,Coffee Shop,601 Union,47.610529,-122.331964,98101,WA
5,Victrola Coffee Roasters,Coffee Shop,1600 3rd Ave,47.610895,-122.338952,98101,WA
6,Cherry Street Coffee,Coffee Shop,509 Olive Way,47.612932,-122.337017,98101,WA
7,Voxx Coffee,Café,1200 6th Ave,47.608919,-122.332504,98101,WA
8,Made Coffee,Café,1531 7th Ave,47.612279,-122.334399,98101,WA
10,Coffee Tree,Coffee Shop,905 8th Ave,47.607388,-122.328003,98104,WA
11,Cherry Street Coffee House,Coffee Shop,320 E Pine St,47.615267,-122.327288,98122,WA
13,Ghost Note Coffee,Coffee Shop,1623 Bellevue Ave,47.615920,-122.326912,98122,WA


In [21]:
df_Cof = clean_cof_dataframe3[clean_cof_dataframe3.categories == 'Coffee Shop']
df_Cof

,name,categories,address,lat,lng,postalCode,state
0,Anchorhead Coffee Co,Coffee Shop,1600 7th Ave Ste 105,47.613450,-122.334702,98101,WA
1,Urban Coffee House,Coffee Shop,1900 4th Ave,47.612460,-122.338748,98101,WA
2,Tully's Coffee - Two Union,Coffee Shop,601 Union,47.610529,-122.331964,98101,WA
5,Victrola Coffee Roasters,Coffee Shop,1600 3rd Ave,47.610895,-122.338952,98101,WA
6,Cherry Street Coffee,Coffee Shop,509 Olive Way,47.612932,-122.337017,98101,WA
10,Coffee Tree,Coffee Shop,905 8th Ave,47.607388,-122.328003,98104,WA
11,Cherry Street Coffee House,Coffee Shop,320 E Pine St,47.615267,-122.327288,98122,WA
13,Ghost Note Coffee,Coffee Shop,1623 Bellevue Ave,47.615920,-122.326912,98122,WA
14,Starbucks Reserve Roastery,Coffee Shop,1124 Pike St,47.614044,-122.328289,98101,WA
15,Starbucks,Coffee Shop,400 Pine St,47.611447,-122.337502,98101,WA


## Generate the list of the hotels, restaurants and coffee shops near the WSCC.

In [22]:
# create dataframe of the neighbourhood
park_neighbourhood_df = pd.concat([df_Hotel, df_Restaurant, df_Cof], ignore_index=True)
park_neighbourhood_df.sort_values('postalCode', inplace=True)
park_neighbourhood_df

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Seattle Hotel,Hotel,1400 6th Ave,47.610697,-122.333895,98101,WA
27,Thai Garden Restaurant,Thai Restaurant,400 Pine St,47.611256,-122.337571,98101,WA
55,Starbucks,Coffee Shop,600 Stewart St,47.613830,-122.336876,98101,WA
29,Dragonfish Asian Cafe,Asian Restaurant,722 Pine St,47.613019,-122.333437,98101,WA
30,Nikko Restaurant,Japanese Restaurant,1900 5th Ave,47.613300,-122.337997,98101,WA
32,Urbane,New American Restaurant,1639 8th Ave,47.613924,-122.334158,98101,WA
33,Il Fornaio Seattle,Italian Restaurant,600 Pine St,47.613027,-122.336302,98101,WA
34,Barolo Ristorante,Italian Restaurant,1940 Westlake Ave,47.614298,-122.337838,98101,WA
37,Ristorante Machiavelli,Italian Restaurant,1215 Pine St,47.615092,-122.328125,98101,WA
38,Anchorhead Coffee Co,Coffee Shop,1600 7th Ave Ste 105,47.613450,-122.334702,98101,WA


In [23]:

# Generate map to visualize park neighbourhood including Restaurant and Cafeteria 
location_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(park_neighbourhood_df['lat'], park_neighbourhood_df['lng'], 
                                           park_neighbourhood_df['name'], park_neighbourhood_df['categories'],\
                                               park_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(location_map)  
    
location_map
 

## Step 2: Let explore more about Italian restaurant in our dataframe

## Italian restaurants in Seattle ~500m to WSCC

In [24]:
# search for Restaurants
search_query = 'Italian'
radius = 5000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=NJBI1NENJYENG0SGJTTS1RFHOKSQEHBMWXQHPRLSLI0XU0OX&client_secret=E5GJDJ4JT02EDEH0PVGKF1ZN0NJCRTLXPJFMYPP0OJ4XAKLA&ll=47.61175385,-122.331562750662&v=20180604&query=Italian&radius=5000&limit=30'

In [25]:
# Send the GET Request and examine the results
Rresults = requests.get(url).json()
#Rresults

# assign relevant part of JSON to venues
venues = Rresults['response']['venues']

# tranform venues into a dataframe
Italian_dataframe = json_normalize(venues)
Italian_dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,...,location.state,location.country,location.formattedAddress,venuePage.id,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name
0,57fef05238fa20e89cb7711d,Luigi's Italian Eatery,"[{'id': '56aa371be4b08b9a8d573538', 'name': 'T...",v-1565642141,False,625 1st Ave,Cherry,47.602459,-122.334352,"[{'label': 'display', 'lat': 47.60245929546312...",...,WA,United States,"[625 1st Ave (Cherry), Seattle, WA 98104, Unit...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4e67c741c65be127ef0f5a0e,Reza's Italian Cafe,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1565642141,False,1220 Howell St,Yale,47.617983,-122.330145,"[{'label': 'display', 'lat': 47.61798250120272...",...,WA,United States,"[1220 Howell St (Yale), Seattle, WA 98101, Uni...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,577723e4498e0572192491d2,Italian Family Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1565642141,False,1208 Madison St,NaN,47.609348,-122.325302,"[{'label': 'display', 'lat': 47.60934802201264...",...,WA,United States,"[1208 Madison St, Seattle, WA 98104, United St...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4f8f71bae4b0736826c64e1e,Italian Bistro,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1565642141,False,NaN,NaN,47.591885,-122.333774,"[{'label': 'display', 'lat': 47.59188501272685...",...,WA,United States,"[Seattle, WA 98134, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4f3236b919836c91c7c15bca,Amicis Family Italian Restaurant,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1565642141,False,1106 8th Ave,NaN,47.608721,-122.329190,"[{'label': 'display', 'lat': 47.608721, 'lng':...",...,WA,United States,"[1106 8th Ave, Seattle, WA 98101, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# keep only columns that include venue name, and anything that is associated with location
Italian_clean_columns = ['name', 'categories'] + [col for col in Italian_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Italian_dataframe = Italian_dataframe.loc[:,Italian_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']

# filter the category for each row
clean_Italian_dataframe['categories'] = clean_Italian_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Italian_dataframe.columns = [column.split('.')[-1] for column in clean_Italian_dataframe.columns]

clean_Italian_dataframe.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Luigi's Italian Eatery,Theme Restaurant,625 1st Ave,Cherry,47.602459,-122.334352,"[{'label': 'display', 'lat': 47.60245929546312...",1055,98104,US,Seattle,WA,United States,"[625 1st Ave (Cherry), Seattle, WA 98104, Unit...",57fef05238fa20e89cb7711d
1,Reza's Italian Cafe,Pizza Place,1220 Howell St,Yale,47.617983,-122.330145,"[{'label': 'display', 'lat': 47.61798250120272...",701,98101,US,Seattle,WA,United States,"[1220 Howell St (Yale), Seattle, WA 98101, Uni...",4e67c741c65be127ef0f5a0e
2,Italian Family Pizza,Pizza Place,1208 Madison St,NaN,47.609348,-122.325302,"[{'label': 'display', 'lat': 47.60934802201264...",540,98104,US,Seattle,WA,United States,"[1208 Madison St, Seattle, WA 98104, United St...",577723e4498e0572192491d2
3,Italian Bistro,Italian Restaurant,NaN,NaN,47.591885,-122.333774,"[{'label': 'display', 'lat': 47.59188501272685...",2217,98134,US,Seattle,WA,United States,"[Seattle, WA 98134, United States]",4f8f71bae4b0736826c64e1e
4,Amicis Family Italian Restaurant,Italian Restaurant,1106 8th Ave,NaN,47.608721,-122.329190,"[{'label': 'display', 'lat': 47.608721, 'lng':...",381,98101,US,Seattle,WA,United States,"[1106 8th Ave, Seattle, WA 98101, United States]",4f3236b919836c91c7c15bca


In [27]:
# delete unnecessary columns
clean_Italian_dataframe2= clean_Italian_dataframe.drop(['cc', 'city', 'country', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id', 'crossStreet'], axis=1)
clean_Italian_dataframe2


,name,categories,address,lat,lng,postalCode,state
0,Luigi's Italian Eatery,Theme Restaurant,625 1st Ave,47.602459,-122.334352,98104,WA
1,Reza's Italian Cafe,Pizza Place,1220 Howell St,47.617983,-122.330145,98101,WA
2,Italian Family Pizza,Pizza Place,1208 Madison St,47.609348,-122.325302,98104,WA
3,Italian Bistro,Italian Restaurant,NaN,47.591885,-122.333774,98134,WA
4,Amicis Family Italian Restaurant,Italian Restaurant,1106 8th Ave,47.608721,-122.329190,98101,WA
5,That's Amore Italian Cafe,Italian Restaurant,1425 31st Ave S,47.589577,-122.292576,98144,WA
6,WiseGuy Italian Street Food,Food Truck,Mobile,47.622078,-122.337258,NaN,WA
7,Gelatiamo,Dessert Shop,1400 3rd Ave,47.608754,-122.336992,98101,WA
8,DeLaurenti Specialty Food & Wine,Wine Shop,1435 1st Ave,47.608781,-122.340113,98101,WA
9,Bizzarro Italian Cafe,Italian Restaurant,1307 N 46th St,47.662043,-122.341671,98103,WA


In [28]:
# delete rows with none values
df_Italian = clean_Italian_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Italian

,name,categories,address,lat,lng,postalCode,state
0,Luigi's Italian Eatery,Theme Restaurant,625 1st Ave,47.602459,-122.334352,98104,WA
1,Reza's Italian Cafe,Pizza Place,1220 Howell St,47.617983,-122.330145,98101,WA
2,Italian Family Pizza,Pizza Place,1208 Madison St,47.609348,-122.325302,98104,WA
4,Amicis Family Italian Restaurant,Italian Restaurant,1106 8th Ave,47.608721,-122.329190,98101,WA
5,That's Amore Italian Cafe,Italian Restaurant,1425 31st Ave S,47.589577,-122.292576,98144,WA
7,Gelatiamo,Dessert Shop,1400 3rd Ave,47.608754,-122.336992,98101,WA
8,DeLaurenti Specialty Food & Wine,Wine Shop,1435 1st Ave,47.608781,-122.340113,98101,WA
9,Bizzarro Italian Cafe,Italian Restaurant,1307 N 46th St,47.662043,-122.341671,98103,WA
10,Calozzi's Italian Kitchen,American Restaurant,90 Blanchard St,47.612246,-122.345525,98121,WA
11,Cucina De Ra Italian Restaurant,Food,2801 Western Ave,47.615703,-122.352583,98121,WA


In [29]:
from sklearn.cluster import KMeans

number_of_clusters = 5

xys = df_Italian[['lat', 'lng']].values
k_means = KMeans(n_clusters=number_of_clusters, random_state=0).fit(xys)
k_means_cluster_centers = k_means.cluster_centers_
k_means_cluster_centers

array([[  47.62373833, -122.33605174],
       [  47.554695  , -122.32254567],
       [  47.60729219, -122.33292154],
       [  47.65572096, -122.34522816],
       [  47.58957685, -122.29257637]])

In [30]:
map_italian = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng in k_means_cluster_centers:
    folium.Circle([lat, lng], radius=200, color='green', fill=True, fill_color='green', fill_opacity=0.9).add_to(map_italian) 
for lat, lng in zip(df_Italian['lat'], df_Italian['lng']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_italian)  

map_italian



# 5.Results

My analysis shows that there is a great number of restaurants, coffee shops and hotels around Seattle Convention Center (~500 meters from WSCC). The list of restaurants, hotels and coffee shops will help the event planner to make consideration about the schedule of the event. From the map, we can see the hign  concentration of restaurants and hotels were detected west of I5, in west lake neighborhood. 

For Italian shops(including restaurant, coffee shop, wine and desert shop), I focused on around 5km from WSCC. Those location candidates were then clustered to create zones of interest which contain greatest number of location candidates. Result of all this is 5 zones containing largest number of potential new restaurant locations based on number of and distance to existing venues. This does not imply that those zones are actually optimal locations for participant from Italy. From the map, I would recommend the zone on the west of WSCC. There are low Italian shop density fairly close to WSCC.

# 6.Discussion

Purpose of this project was to identify the list of hotels, restaurants and coffee shops which are close to Washington State Convention Center for the event planner. Particularly, most of the people were from Italy, I provided the analysis about Italian community near WSCC. By calculating hotel, restaurant and coffee shop density distribution from Foursquare data I have first identified the location map for recommendation. Clustering of those Italian shop locations was then performed in order to create major zones of interest.

Final decission on optimal location will be made by the event planner. Based on my analysis, I will recommend the region on the west of the I5 with high density of hotels and restaurants and specific characteristics of Italian neighborhoods.